## Start by setting week number
#### Wait for ESPN data update until at least Wednesday morning, but run latest by Thursday night

https://www.espn.com/nfl/stats/team/_/stat/total/season/2022/seasontype/2

In [1]:
SET_WEEK = 2
week_name = 'week_' + str(SET_WEEK)
print(week_name)

week_2


In [2]:
from scrapers.scrape_2022 import get_2022_data
import pandas as pd
import numpy as np
import pickle

import warnings
warnings.filterwarnings('ignore')

### Models

In [3]:
logit = '../modeling/logit.pkl'
rf = '../modeling/rf.pkl'
gbm = '../modeling/gbm.pkl'
lgb = '../modeling/lgb.pkl'
catboost = '../modeling/catboost.pkl'

with open(logit, 'rb') as file:
    LOGIT = pickle.load(file)

with open(rf, 'rb') as file:
    RF = pickle.load(file)
    
with open(gbm, 'rb') as file:
    GBM = pickle.load(file)
    
with open(lgb, 'rb') as file:
    LGB = pickle.load(file)
    
with open(catboost, 'rb') as file:
    CATBOOST= pickle.load(file)

### 2022 data

In [4]:
%%time

X = get_2022_data()

# from boruta elimination method:
keep_cols = ['offense_points_per_game',
             'offense_downs_Fourth Downs_PCT',
             'offense_passing_AVG',
             'offense_passing_YDS/G',
             'offense_passing_RTG',
             'offense_receiving_AVG',
             'defense_points_per_game',
             'defense_passing_AVG',
             'defense_receiving_AVG',
             'defense_rushing_YDS/G',
             'defense_passing_SYL_per_game',
             'offense_downs_Third Downs_ATT_per_game',
             'defense_downs_Third Downs_ATT_per_game',
             'defense_downs_First Downs_penalty_ratio',
             'offense_passing_TD_per_game',
             'defense_rushing_TD_per_game',
             'defense_pass_TD_per_rush_TD',
             'offense_pass_TD_to_INT']

X = X[['Team', 'games_played', 'season'] + keep_cols].copy()

X.head(2)

Wall time: 30.7 s


,Team,games_played,season,offense_points_per_game,offense_downs_Fourth Downs_PCT,offense_passing_AVG,offense_passing_YDS/G,offense_passing_RTG,offense_receiving_AVG,defense_points_per_game,...,defense_receiving_AVG,defense_rushing_YDS/G,defense_passing_SYL_per_game,offense_downs_Third Downs_ATT_per_game,defense_downs_Third Downs_ATT_per_game,defense_downs_First Downs_penalty_ratio,offense_passing_TD_per_game,defense_rushing_TD_per_game,defense_pass_TD_per_rush_TD,offense_pass_TD_to_INT
0,Philadelphia Eagles,2,2022,31.0,66.7,9.1,281.0,97.1,13.1,21.0,...,9.1,121.5,14.5,15.0,13.0,0.069767,0.5,1.5,1.0,1.0
1,Miami Dolphins,2,2022,31.0,100.0,8.9,351.5,116.5,12.5,22.5,...,12.6,116.5,10.0,12.5,9.5,0.150000,3.5,0.5,4.0,3.5


In [5]:
print(X.shape)

(32, 21)


In [6]:
X.fillna(0, inplace = True)
X.replace([np.inf, -np.inf], 0, inplace=True)

In [7]:
X['games_played'].unique().tolist()

[2]

In [8]:
# X[X['games_played'] == 8]

In [9]:
predictions = X[['Team', 'season']].copy()
predictions['week_marker'] = week_name

predictions['pred_logit'] = LOGIT.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_rf'] = RF.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_gbm'] = GBM.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_lgb'] = LGB.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_catboost'] = CATBOOST.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_mean'] = (predictions['pred_logit'] + predictions['pred_rf'] + predictions['pred_gbm'] + predictions['pred_lgb'] + predictions['pred_catboost']) / 5
predictions['pred_opt_combo'] = (predictions['pred_logit'] + 7*predictions['pred_catboost']) / 8

predictions.sort_values('pred_opt_combo', ascending = False, inplace = True)
predictions.head(3)

,Team,season,week_marker,pred_logit,pred_rf,pred_gbm,pred_lgb,pred_catboost,pred_mean,pred_opt_combo
2,Buffalo Bills,2022,week_2,0.997101,0.382366,0.241442,0.362338,0.419276,0.480505,0.491504
4,Kansas City Chiefs,2022,week_2,0.267241,0.260254,0.198541,0.337829,0.426081,0.297989,0.406226
23,Tampa Bay Buccaneers,2022,week_2,0.598620,0.165052,0.106428,0.206401,0.360522,0.287405,0.390284


In [10]:
predictions.to_csv('weekly_runs_2022/' + week_name + '.csv', index = False)

### Check weekly run

In [15]:
predictions

,Team,season,week_marker,pred_logit,pred_rf,pred_gbm,pred_lgb,pred_catboost,pred_mean,pred_opt_combo
6,Buffalo Bills,2022,week_1,9.994453e-01,0.371555,0.226497,0.382055,0.412690,0.478448,0.486035
1,Kansas City Chiefs,2022,week_1,9.719945e-01,0.312390,0.213583,0.358647,0.408618,0.453047,0.479041
17,Tampa Bay Buccaneers,2022,week_1,9.839706e-01,0.182806,0.119485,0.289598,0.372766,0.389725,0.449166
24,Baltimore Ravens,2022,week_1,8.532703e-01,0.264963,0.180135,0.372077,0.391360,0.412361,0.449099
27,Carolina Panthers,2022,week_1,5.856759e-01,0.179008,0.105350,0.232223,0.361512,0.292754,0.389533
9,Washington Commanders,2022,week_1,2.969123e-01,0.161451,0.097715,0.233803,0.365586,0.231093,0.357002
7,Minnesota Vikings,2022,week_1,9.002089e-02,0.310272,0.155652,0.289348,0.389651,0.246989,0.352198
0,Indianapolis Colts,2022,week_1,1.822013e-01,0.029447,0.066750,0.200666,0.361633,0.168139,0.339204
8,New York Giants,2022,week_1,1.156594e-04,0.175252,0.077953,0.254964,0.379737,0.177604,0.332285
28,Seattle Seahawks,2022,week_1,3.965890e-04,0.166231,0.110410,0.286142,0.379068,0.188450,0.331734
